In [1]:
import pickle as pkl
from time import time
import scipy.io as sio

search=False

from bonsai.data_loaders import load_data
from bonsai.net import Net
from bonsai.trainers import *
from bonsai.helpers import cell_cnx, show_time
from bonsai.ops import commons

%load_ext autoreload
%autoreload 2

# Load Meta

In [22]:
class_mat = sio.loadmat('data/ImageNet/devkit/data/meta.mat')
classes = {}
for in_class in class_mat['synsets']:
    class_ = {'name':in_class[0][2][0],'desc':in_class[0][3][0]}
    classes[in_class[0][1][0]] = class_
classes

{'n02119789': {'name': 'kit fox, Vulpes macrotis',
  'desc': 'small grey fox of southwestern United States; may be a subspecies of Vulpes velox'},
 'n02100735': {'name': 'English setter',
  'desc': 'an English breed having a plumed tail and a soft silky coat that is chiefly white'},
 'n02110185': {'name': 'Siberian husky',
  'desc': 'breed of sled dog developed in northeastern Siberia; they resemble the larger Alaskan malamutes'},
 'n02096294': {'name': 'Australian terrier',
  'desc': 'small greyish wire-haired breed of terrier from Australia similar to the cairn'},
 'n02102040': {'name': 'English springer, English springer spaniel',
  'desc': 'a breed having typically a black-and-white coat'},
 'n02066245': {'name': 'grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus',
  'desc': 'medium-sized greyish-black whale of the northern Pacific'},
 'n02509815': {'name': 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens',
  'desc': 'reddish-brown 

# Setup

In [3]:
lr_schedule = {'lr_min': 0,
               'lr_max': 1e-2,
               't_0': 2,
               't_mult': 1}

batch_size = 64
dataset = 'ImageNet'
data = load_data(batch_size,dataset,metadata={'classes':1000})

Generating ImageNet metadata: data_shape


# Search

In [26]:
if search:
    batch_size = 64
    scale = 2
    reductions = 5
    spacing = 3 
    nodes = 4 
    data = load_data(batch_size, dataset)
    print("Edges per cell:",cell_cnx(nodes))
    
    model = Net(dim=data[2], classes=data[3], scale=scale, reductions=reductions, spacing=spacing, nodes=nodes, auxiliary=True)
    model.save_genotype()
    print(model)

# Train

In [6]:
data = load_data(64, dataset)
params, genotype = pkl.load(open('genotype.pkl','rb'))
params['genotype']=genotype
params['scale']=7
params['prune']=False
params['auxiliary']=True
params['dim']=data[2]
model = Net(**params)
model.detail_print()

lr_schedule = {'lr_min': 0,
               'lr_max': 1e-2,
               't_0': 600,
               't_mult': 1}

==================== NETWORK ===================
Dim: [56, 3, 32, 32], Scale: 6, Spacing: 3, Reductions: 5, Nodes: 6, Pruners: False, Aux: True
Initializer              :      1,856 params
=== Cell 0  (Normal)     :     52,032 params ===
    x:    [0, 1, 2, 3, 4, 5]
    y:    [0, 1, 2, 4, 5]
    0->1: ['Sep_Conv_3x3']
    0->2: ['Identity']
    0->3: ['Sep_Conv_3x3']
    0->4: ['Max_Pool_3x3']
    0->5: ['Identity', 'Max_Pool_3x3']
    1->2: ['Sep_Conv_3x3']
    1->3: ['Conv_1x1']
    1->4: ['Max_Pool_3x3']
    1->5: ['Max_Pool_3x3']
    2->3: ['Identity']
    2->4: ['Sep_Conv_3x3']
    2->5: ['Conv_1x1']
    3->4: ['Max_Pool_3x3']
    3->5: ['Conv_1x1']
    4->5: ['Sep_Conv_3x3']
=== Cell 1  (Normal)     :     32,448 params ===
    x:    [0, 1, 2, 4, 5]
    y:    [0, 1, 2, 3, 5]
    0->1: ['Sep_Conv_3x3']
    0->2: ['Max_Pool_3x3']
    0->3: ['Sep_Conv_3x3']
    0->4: ['Max_Pool_3x3']
    0->5: ['Max_Pool_3x3']
    1->2: ['Max_Pool_3x3']
    1->3: ['Max_Pool_3x3']
    1->4: []
    1->

In [7]:
full_train(
    model,
    data,
    epochs=512,
    drop_prob=.3,
    lr_schedule=lr_schedule
);

0 211.25MiB
1 1.97GiB
2 3.17GiB
3 4.55GiB
4 4.95GiB
5 5.65GiB
6 6.24GiB
7 6.55GiB
8 6.84GiB
9 7.24GiB
10 7.37GiB
11 7.52GiB
12 7.72GiB
13 7.82GiB
14 7.92GiB
Train Epoch: 0   [49952 /50000  (100%)]	Loss: 2.599, Time: 0.452s, Per Epoch: 6m,43s     , Alloc: 8.97GiB  
Train Corrects: 36.14% Time: 6m,33s
Test Corrects:  55.99%, 21.83 s

Adjusting lr to 0.01


KeyboardInterrupt: 